### Packages

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import numpy.random as nr
import itertools as it
import operator as op
import random as rd

### Input

In [2]:
N = int(input('Input number of airports'))

Input number of airports 12000


### Data

In [3]:
toc = dt.datetime.now() #program start time

nr.seed(20220606)
rd.seed(20220606)

#flight costs matrix
C = 708 + (51124 - 708)*nr.rand(N**2).reshape(N,N)
C -= np.diag(np.diag(C)) 
        
interval = {}
interval["Acheck"] = nr.randint(56, 71)
interval["Bcheck"] = nr.randint(180, 241)
interval["Ccheck"] = nr.randint(540, 721)
interval["Dcheck"] = nr.randint(2160, 3601)
        
durations = {}
durations["Acheck"] = [1]*N
durations["Bcheck"] = nr.randint(1, 4, size = N)
durations["Ccheck"] = nr.randint(7, 15, size = N)
durations["Dcheck"] = nr.randint(21, 43, size = N)
        
costs = {}
costs["Acheck"] = (310 + (820 - 310)*nr.rand(N))
costs["Bcheck"] = (4960 + (7380 - 4960)*nr.rand(N))
costs["Ccheck"] = (186000 + (246000 - 186000)*nr.rand(N))
costs["Dcheck"] = (930000 + (2050000 - 930000)*nr.rand(N))

MFPD = 4 #maximum flights per day
home = nr.randint(N) #initial airport
I = set(np.arange(N)) #airport set
MFCPD = C.max() #maximum flight costs per day
MFC = (C + np.diag([np.inf]*N)).min() #minimum flight costs
MDP = (MFCPD + MFPD*MFC)/2 #median daily profit

### Flight Simulation

In [4]:
def nearest_neighbor(branch, route):
    for airport in route:
        branch[airport] = np.inf
        
    return np.argmin(branch)

In [5]:
def maintenance_check (airport, day, maintenance, route, FCPD):
    branchP = np.zeros(N)
    for check in maintenance["amount"].keys():
                
        if day >= (maintenance["amount"][check]+1)*interval[check] :        
            branchP += costs[check][:].copy() + np.array(durations[check][:].copy())*MDP
        
    #when maintenance check occurs
    if sum(branchP) >0:
            
        branch = branchP + C[route[-2]][:].copy()
        airport = nearest_neighbor(branch, route[:-1])
        
        for check in maintenance["amount"].keys():
            if day >= (maintenance["amount"][check]+1)*interval[check] :
            
                maintenance["amount"][check] += 1
                day += durations[check][airport]
            
                maintenance["cost"][check] += costs[check][airport]
                maintenance["loss"][check] += durations[check][airport]*MDP
        
        FCPD = FCPD - C[route[-2]][route[-1]] + C[route[-2]][airport]    #replacement
        route = route[:-1]  + (airport,)      
        
        branch = C[airport][:].copy()
        airport = nearest_neighbor(branch, route)
        
    return airport, route, FCPD, day, maintenance

### Main Program

In [6]:
route = (home,)

day = 0 # aircraft age
        
maintenance = {}
maintenance["loss"]     = {"Acheck" : 0, "Bcheck" : 0, "Ccheck" : 0, "Dcheck" : 0}
maintenance["amount"]   = {"Acheck" : 0, "Bcheck" : 0, "Ccheck" : 0, "Dcheck" : 0}
maintenance["cost"]     = {"Acheck" : 0, "Bcheck" : 0, "Ccheck" : 0, "Dcheck" : 0}
        
FC = 0 #flight cost
    
FPD = 0 #flights per day
FCPD = 0 #flight costs per day

while (set(route) != I):
    #flight preparation stage
    branch = C[route[-1]][:].copy()
    airport = nearest_neighbor(branch, route)
    
    #flight stage
    FCPD += C[route[-1]][airport]
    FPD += 1
     
    #when it reaches the maximum limit per day
    if (FPD > MFPD) or (FCPD > MFCPD):
        
        #step back so as not to reach the maximum limit per day
        FCPD -= C[route[-1]][airport] 
        day += 1 
        
        #maintenance check stage
        airport, route, FCPD, day, maintenance = maintenance_check (airport, day, maintenance, route, FCPD)
        FC += FCPD  
        
        FCPD = C[route[-1]][airport]
        FPD = 1
    
    route = route + (airport,)
        
FCPD += C[route[-1]][home]
FPD +=1
route = route + (home,)
#when it reaches the maximum limit per day
if (FPD > MFPD) or (FCPD > MFCPD):
        
    #step back so as not to reach the maximum limit per day
    FCPD -= C[route[-1]][airport] 
    day += 1 
        
    #maintenance check stage
    airport, route, FCPD, day, maintenance = maintenance_check (airport, day, maintenance, route, FCPD)
    FC += FCPD  

    FCPD = C[route[-1]][airport]
    FPD = 1
    
FC += FCPD + C[route[-1]][home]    
day += 1

objektif = FC + sum(list(maintenance["cost"].values()))
objektif += sum(list(maintenance["loss"].values()))

In [7]:
def output():
    print('Total cost of flights  $%d' 
        %(FC))
    print('Total maintenance costs $%d' 
        %(sum(list(maintenance["cost"].values()))))
    print('Total maintenance loss $%d' 
        %(sum(list(maintenance["loss"].values()))))

    print('Total cost $%d' 
        %(FC + sum(list(maintenance["cost"].values()))))
    print()
    print("\033[1m" +"Details :" +"\033[0m")
    print(f'flight duration {int(day)} day')
    print()
    print('Number of The A check maintenance = %d' 
        %(maintenance["amount"]['Acheck']))
    print('Number of The B check maintenance = %d' 
        %(maintenance["amount"]['Bcheck']))
    print('Number of The C check maintenance = %d' 
        %(maintenance["amount"]['Ccheck']))
    print('Number of The D check maintenance = %d' 
        %(maintenance["amount"]['Dcheck']))
    print()
    print('Costs of The A check maintenance = $%d' 
        %(maintenance["cost"]['Acheck']))
    print('Costs of The B check maintenance = $%d' 
        %(maintenance["cost"]['Bcheck']))
    print('Costs of The C check maintenance = $%d' 
        %(maintenance["cost"]['Ccheck']))
    print('Costs of The D check maintenance = $%d' 
        %(maintenance["cost"]['Dcheck']))
    print()
    print('Loss of The A check maintenance = $%d' 
        %(maintenance["loss"]['Acheck']))
    print('Loss of The B check maintenance = $%d' 
        %(maintenance["loss"]['Bcheck']))
    print('Loss of The C check maintenance = $%d' 
        %(maintenance["loss"]['Ccheck']))
    print('Loss of The D check maintenance = $%d' 
        %(maintenance["loss"]['Dcheck']))
    print()

    print("route : ")
    for airport in route[:-1]:
        print(f'{airport} -> ', end ='')
       
    print(home)
    print()
    print("Execution time : ", dt.datetime.now()-toc)

def detail():
    print('Detail: ')
    FC = 0
    FC_per_day = 0
    flight_per_day = 0 
    day = 0
    for i in range(1,len(route)):
        FC_per_day += C[route[i-1]][route[i]]
        flight_per_day += 1
    
        if (flight_per_day > MFPD) or (FC_per_day > MFCPD):
            FC += (FC_per_day - C[route[i-1]][route[i]])
            day += 1
        
            print('num: '+str(day)+', airport visited: '+str(route[i-flight_per_day :i])+', flight costs: $'+str(FC_per_day - C[route[i-1]][route[i]]))
            FC_per_day = C[route[i-1]][route[i]]
            flight_per_day = 1
        
    FC += FC_per_day
    day += 1
        
    print('num: '+str(day)+', airport visited: '+str(route[-flight_per_day-1 :])+', flight costs: $'+str(FC_per_day))
    print()
    print("Total cost of flights: $"+str(int(FC)))

### Output

In [8]:
output()

Total cost of flights  $8973280
Total maintenance costs $1782298
Total maintenance loss $3102470
Total cost $10755578

Details :
flight duration 3116 day

Number of The A check maintenance = 51
Number of The B check maintenance = 14
Number of The C check maintenance = 4
Number of The D check maintenance = 1

Costs of The A check maintenance = $18054
Costs of The B check maintenance = $70981
Costs of The C check maintenance = $751666
Costs of The D check maintenance = $941595

Loss of The A check maintenance = $1375878
Loss of The B check maintenance = $377692
Loss of The C check maintenance = $755384
Loss of The D check maintenance = $593516

route : 
1778 -> 11273 -> 3885 -> 11586 -> 3031 -> 5506 -> 8218 -> 271 -> 3088 -> 3232 -> 7013 -> 10116 -> 11515 -> 8154 -> 2843 -> 7189 -> 4198 -> 1658 -> 11938 -> 3752 -> 1440 -> 6459 -> 10944 -> 3561 -> 9777 -> 4611 -> 1484 -> 10438 -> 11034 -> 10455 -> 10947 -> 5479 -> 10810 -> 10420 -> 10631 -> 10958 -> 7559 -> 5729 -> 9105 -> 8484 -> 8521 ->

### Detail

In [9]:
detail()

Detail: 
num: 1, airport visited: (1778, 11273, 3885, 11586, 3031), flight costs: $2854.6345463018774
num: 2, airport visited: (3031, 5506, 8218, 271, 3088), flight costs: $2846.518452402727
num: 3, airport visited: (3088, 3232, 7013, 10116, 11515), flight costs: $2840.0005942489934
num: 4, airport visited: (11515, 8154, 2843, 7189, 4198), flight costs: $2856.743315424774
num: 5, airport visited: (4198, 1658, 11938, 3752, 1440), flight costs: $2853.4521686857365
num: 6, airport visited: (1440, 6459, 10944, 3561, 9777), flight costs: $2855.259815804944
num: 7, airport visited: (9777, 4611, 1484, 10438, 11034), flight costs: $2853.6693560694403
num: 8, airport visited: (11034, 10455, 10947, 5479, 10810), flight costs: $2851.3593352763955
num: 9, airport visited: (10810, 10420, 10631, 10958, 7559), flight costs: $2840.434480435572
num: 10, airport visited: (7559, 5729, 9105, 8484, 8521), flight costs: $2845.518534426885
num: 11, airport visited: (8521, 2589, 9462, 2135, 4469), flight cost